## Necesary conditions for Jobsequencing
### Install pyQUBO from Recruit Communications Co. Ltd.
    pip install pyqubo
### Install openJij from Jij Inc.  (startup from Tohoku University)
    pip install -U cmake (in google collaboratory, update cmake)
    pip install open jij
### Install dwave-ocean-sdk from D-Wave Systems Inc.
    pip install dwave-ocean-sdk
    dwave config create
    Profile (create new) [prod]: prod
    API endpoint URL [skip]: (Enter)
    Authentication token [skip]:
    Default client class (qpu or sw) [qpu]: (Enter)
    Default solver [skip]: (Enter)

# d-wave solves number partition

### import pyQUBO, openJij and numpy

In [39]:
from pyqubo  import Array,Constraint, Placeholder
import openjij as jij
import numpy as np

Array, Constrains and Placeholders are convenient classes from pyQUBO

### Prepare spin variables

In [60]:
N = 8
vartype = "SPIN"
x = Array.create("x",shape=N,vartype=vartype)

"x" is name of variables  
shape specifies the shape of variables as vector, matrix, or...  
vartype selects -1 or 1 by "SPIN" and 0 or 1by "BINARY"

In [61]:
print(x)

Array([Spin(x[0]), Spin(x[1]), Spin(x[2]), Spin(x[3]), Spin(x[4]), Spin(x[5]), Spin(x[6]), Spin(x[7])])


### Let generate integers

In [62]:
n = np.linspace(1,N,N)

In [63]:
print(n)

[1. 2. 3. 4. 5. 6. 7. 8.]


### Define cost function

In [64]:
E_cost = (np.sum(n*x))**2

### Compile the cost function

In [65]:
model = E_cost.compile()

### Get qubo matrix

In [66]:
h,J, offset = model.to_ising()

### Prepare quantum annealer

In [67]:
from dwave.system.samplers import DWaveSampler
sampler = DWaveSampler(solver="DW_2000Q_VFYC_5",token="c001f3c1983c837ecb9d48f4db8b71a9ffb2d10")
V = sampler.nodelist
A = sampler.edgelist

V is a node set called from the current situation in the D-Wave 2000Q.  
A is an edge set called from the current situation in the D-Wave 2000Q.

### Find embedding on the chimera graph

In [68]:
adjacency = {}
for i in J.keys():
    adjacency[i] = 1

Here adjacency detects non-zero interactions in the matrix J.

In [69]:
from minorminer import find_embedding 
embedding = find_embedding(adjacency, A)

In [70]:
print(embedding)

{'x[0]': [932, 930, 940], 'x[1]': [1056, 928, 1063, 1071], 'x[2]': [1064, 1069, 936], 'x[3]': [942, 929, 934], 'x[4]': [935, 943], 'x[5]': [1068, 1067, 939], 'x[6]': [941, 938, 933], 'x[7]': [1065, 937]}


### Embed the problem onto  the chimera graph

In [71]:
from dwave.system.composites import FixedEmbeddingComposite
sampler = FixedEmbeddingComposite(sampler, embedding=embedding)

### Perform quantum annealing by D-Wave 2000Q

In [72]:
response =  sampler.sample_ising(h,J, postprocess="optimization",num_reads=1000)

postprocess has three choices as "optimization", "sampling", and "None".  
num_reads denotes the number of iterations of quantum annealing.

### Show results

In [73]:
print(response)

  x[0] x[1] x[2] x[3] x[4] x[5] x[6] x[7] energy num_oc. chain_.
0   +1   -1   -1   +1   +1   -1   +1   -1 -200.0      97    0.75
1   +1   +1   -1   +1   +1   -1   +1   -1 -200.0      24    0.75
2   -1   -1   +1   -1   -1   +1   -1   +1 -200.0     215    0.75
3   -1   +1   +1   -1   -1   +1   -1   +1 -200.0     211    0.75
4   +1   +1   -1   +1   +1   -1   +1   -1 -200.0      67    0.75
5   +1   +1   -1   +1   +1   -1   -1   -1  -60.0     265    0.75
6   -1   -1   +1   -1   +1   +1   +1   +1  280.0     121    0.75
['SPIN', 7 rows, 1000 samples, 8 variables]


### minimum sample

In [74]:
response.record["sample"][0]

array([ 1, -1, -1,  1,  1, -1,  1, -1], dtype=int8)

### decode solution through pyQUBO

In [75]:
dsol, broken, energy = model.decode_solution(response.record["sample"][0], vartype=vartype)

In [76]:
dsol

{'x': {0: 1.0, 1: -1.0, 2: -1.0, 3: 1.0, 4: 1.0, 5: -1.0, 6: 1.0, 7: -1.0}}